Dans ce notebook nous allons analyser la base de donées des post Python et R pour un entrainement de modèles supervisés.

Nous allons commencer avec une regresion logistique pour une clasification entre les posts Python et R, nous allons par la suite ajouter les autres tags disponibles et faire une classifaction One vs All 

Une fois le modèle entrainé nous allons le comparer avec un modèle RNN

# Importation des bibliothèques

In [2]:
import pandas as pd
from nltk.corpus import stopwords
import re
from wordcloud import WordCloud , STOPWORDS
import matplotlib.pyplot as plt
from transformers import (GPT2Config,GPT2LMHeadModel,GPT2Tokenizer)
from tqdm.notebook import tqdm
import torch

pd.set_option('display.max_colwidth', None)

# Importation des données 

In [3]:
dtypes_questions = {'Id':'int32', 'Score': 'int16', 'Title': 'str',
                    'Body': 'str', 'Title_raw': 'str', 'Text': 'str'}

nrows = 1000

df_questions = pd.read_csv('df_questions_fullclean.csv',
                           usecols=dtypes_questions.keys(),
                           encoding = "utf-8",
                           dtype=dtypes_questions,
                           nrows=nrows
                          )

df_tags_list = pd.read_csv('tags_list.csv',
                           usecols=['Id', 'Tags'],
                           encoding = "utf-8",
                           dtype=dtypes_questions,
                           #nrows=nrows # un post peut avoir plusieurs tags
                          )
print(len(df_questions))
display(df_questions.head(5))
display(df_tags_list.head(5))

1000


,Id,Title,Body,Score,Title_raw,Text
0,1544907,melt two column,follow variable data frame type ialt idx idx idx farve farve dm meltd idctypeialt get variable type ialt value idx idx idx farve farve represent really want something like type ialt value value farve farve represent value reason want would like something line color value fall rise edit shane show reshape data via two melt merge strategy seam ill conceive begin one word see shanes solution ggplotdm labeltypesizeialt geomline geomtextdatasubsetdm variableidxhjust size themebw scalexdiscreteexpandc optslegendpositionnone assume need cast figure data structurelisttype structurecl label calvorligere andre strafferetlige rlove andre bedrageri brandstiftelse mv dokumentfalsk rdselslovovertr vrigt rdselsuheld med falsk forklaring forbr mod myndighed freds og rekr leri rv rk bank forretn mv indbrud fritidshuse garager mv indbrud villaer lejligheder mv love vedr og lign love vedr spil bev ring euforiserende stoffer mangler ved veri simpel vold spiritus og promillek rsel trusler tyvbrugstyv af andet tyvbrugstyv af cykel tyvbrugstyv af indr ret tyvbrugstyv af knallert tyveri fra bil mv omgang med hittegods benloven vold mod myndighed class factor ialt cl idx idx idx farve cgreen green green green green green green green red green green green red green green green green farve cred green green red green green green green green green red name ctype ialt idx idx idx farve farve class dataframe rownames cna,26,melt to two variable columns,melt two column follow variable data frame type ialt idx idx idx farve farve dm meltd idctypeialt get variable type ialt value idx idx idx farve farve represent really want something like type ialt value value farve farve represent value reason want would like something line color value fall rise edit shane show reshape data via two melt merge strategy seam ill conceive begin one word see shanes solution ggplotdm labeltypesizeialt geomline geomtextdatasubsetdm variableidxhjust size themebw scalexdiscreteexpandc optslegendpositionnone assume need cast figure data structurelisttype structurecl label calvorligere andre strafferetlige rlove andre bedrageri brandstiftelse mv dokumentfalsk rdselslovovertr vrigt rdselsuheld med falsk forklaring forbr mod myndighed freds og rekr leri rv rk bank forretn mv indbrud fritidshuse garager mv indbrud villaer lejligheder mv love vedr og lign love vedr spil bev ring euforiserende stoffer mangler ved veri simpel vold spiritus og promillek rsel trusler tyvbrugstyv af andet tyvbrugstyv af cykel tyvbrugstyv af indr ret tyvbrugstyv af knallert tyveri fra bil mv omgang med hittegods benloven vold mod myndighed class factor ialt cl idx idx idx farve cgreen green green green green green green green red green green green red green green green green farve cred green green red green green green green green green red name ctype ialt idx idx idx farve farve class dataframe rownames cna
1,195625,time complexity pop element list python,wonder time complexity pop list object python cpython particulary also value affect complexity,55,What is the time complexity of popping elements from list in Python?,time complexity pop element list python wonder time complexity pop list object python cpython particulary also value affect complexity
2,6052631,data monthyear interval,often work date imagine fairly column represent date dataframe simply want dataframe summarize column monthyear use date approach want dataframe feed plot help provide greatly edit reference strtemp dataframe ob variable date posixct format amount num headtemp date amount,62,Aggregate Daily Data to Month/Year intervals,data monthyear interval often work date imagine fairly column represent date dataframe simply want dataframe summarize column monthyear use date approach want dataframe feed plot help provide greatly edit reference strtemp dataframe ob variable date posixct format amount num headtemp date amount
3,8809004,escape roxygen style documentation,let u say comm

,Id,Tags
0,1544907,r
1,1544907,ggplot2
2,1544907,reshape
3,195625,python
4,6052631,datetime


In [ ]:
# création des labels python et r

